In [5]:
import langchain
from langchain import PromptTemplate
import json
from langchain_openai import ChatOpenAI 
from langchain_groq import ChatGroq

In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("your groq api key")

In [7]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [40]:
email_generation_template = """
You are an AI assistant designed to draft emails based on user requests.
Your output MUST be a valid JSON object.
The JSON object must contain five keys: "to", "subject", "body", "SMTP_SERVER" and "SMTP_PORT".
- "to" should be a single email address string (e.g., "recipient@example.com").
- "subject" should be a concise string for the email subject line.
- "body" should be a string containing the full email content and it should be in a valid letter format with line spaces,but it need to be in json format.
- "SMTP_SERVER" should be a single SMTP_SERVER (e.g.,"smtp.gmail.com" For Gmail).
- "SMTP_PORT" should be a single value (e.g., 465  For SSL (Gmail))

Ensure the email body is professional and directly addresses the user's request.
Do NOT include any text outside the JSON object.

Context: {context}
User Request: {user_request}

JSON Output:
"""

In [41]:
EMAIL_PROMPT = PromptTemplate(
    template=email_generation_template,
    input_variables=["user_request", "context"]
)

In [42]:
user_email_request = "Draft an email to my friend bhanu(me23bt021@iitdh.ac.in) to encourage him to improve his bgmi gameplay and participate in bgmi tournaments"
email_context = "My name is KVKR. I am his friend"

In [43]:
formatted_prompt = EMAIL_PROMPT.format(
    user_request=user_email_request,
    context=email_context
)

In [44]:
print(formatted_prompt)


You are an AI assistant designed to draft emails based on user requests.
Your output MUST be a valid JSON object.
The JSON object must contain five keys: "to", "subject", "body", "SMTP_SERVER" and "SMTP_PORT".
- "to" should be a single email address string (e.g., "recipient@example.com").
- "subject" should be a concise string for the email subject line.
- "body" should be a string containing the full email content and it should be in a valid letter format with line spaces,but it need to be in json format.
- "SMTP_SERVER" should be a single SMTP_SERVER (e.g.,"smtp.gmail.com" For Gmail).
- "SMTP_PORT" should be a single value (e.g., 465  For SSL (Gmail))

Ensure the email body is professional and directly addresses the user's request.
Do NOT include any text outside the JSON object.

Context: My name is KVKR. I am his friend
User Request: Draft an email to my friend bhanu(me23bt021@iitdh.ac.in) to encourage him to improve his bgmi gameplay and participate in bgmi tournaments

JSON Outp

In [45]:
llm_response = llm.invoke(formatted_prompt)
llm_output_str = llm_response.content

print("\n--- Raw LLM Output ---")
print(llm_output_str)


--- Raw LLM Output ---
{
  "to": "me23bt021@iitdh.ac.in",
  "subject": "Improving BGMI Gameplay and Tournament Participation",
  "body": "Dear Bhanu,\n\nI hope this email finds you well. I am writing to encourage you to improve your BGMI gameplay and consider participating in upcoming BGMI tournaments.\n\nI have been observing your gameplay, and I believe you have tremendous potential to excel in the game. With dedicated practice and a strategic approach, I am confident that you can enhance your skills and become a formidable player.\n\nParticipating in tournaments will not only provide you with valuable experience but also give you the opportunity to compete with other skilled players, learn from them, and gain recognition in the gaming community.\n\nI would be happy to help you practice and prepare for tournaments. Let's work together to take your gameplay to the next level.\n\nBest regards,\nKVKR",
  "SMTP_SERVER": "smtp.gmail.com",
  "SMTP_PORT": 465
}


In [46]:
import re

In [47]:
cleaned_text = re.sub(r'<think>.*?</think>', '', llm_output_str, flags=re.DOTALL)
cleaned_text = re.sub(r'```json\s*', '', cleaned_text, flags=re.DOTALL)
cleaned_text = re.sub(r'\s*```', '', cleaned_text, flags=re.DOTALL)
cleaned_text = cleaned_text.strip()
cleaned_text = re.sub(r'\s*\n\s*', '\n', cleaned_text)
cleaned_text = re.sub(r' +', ' ', cleaned_text)

In [48]:
print(cleaned_text)

{
"to": "me23bt021@iitdh.ac.in",
"subject": "Improving BGMI Gameplay and Tournament Participation",
"body": "Dear Bhanu,\n\nI hope this email finds you well. I am writing to encourage you to improve your BGMI gameplay and consider participating in upcoming BGMI tournaments.\n\nI have been observing your gameplay, and I believe you have tremendous potential to excel in the game. With dedicated practice and a strategic approach, I am confident that you can enhance your skills and become a formidable player.\n\nParticipating in tournaments will not only provide you with valuable experience but also give you the opportunity to compete with other skilled players, learn from them, and gain recognition in the gaming community.\n\nI would be happy to help you practice and prepare for tournaments. Let's work together to take your gameplay to the next level.\n\nBest regards,\nKVKR",
"SMTP_SERVER": "smtp.gmail.com",
"SMTP_PORT": 465
}


In [49]:
try:
    email_data = json.loads(cleaned_text)

    print("\n--- Parsed Email Data ---")
    print(f"To: {email_data['to']}")
    print(f"Subject: {email_data['subject']}")
    print(f"SMTP_SERVER:\n{email_data['SMTP_SERVER']}")
    print(f"SMTP_PORT:\n{email_data['SMTP_PORT']}")


except json.JSONDecodeError as e:
    print(f"\nError: Failed to parse JSON output from LLM: {e}")
    print("LLM might not have followed the JSON format correctly.")
except KeyError as e:
    print(f"\nError: Missing expected key in JSON output: {e}")


--- Parsed Email Data ---
To: me23bt021@iitdh.ac.in
Subject: Improving BGMI Gameplay and Tournament Participation
SMTP_SERVER:
smtp.gmail.com
SMTP_PORT:
465


In [50]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl 

def send_email_via_smtp(sender_email, receiver_email, subject, body, smtp_server, smtp_port, password):
    """
    Sends an email via SMTP.
    """
    try:
        # Create a multipart message and set headers
        message = MIMEMultipart()
        message["From"] = sender_email
        message["To"] = receiver_email
        message["Subject"] = subject
        message.attach(MIMEText(body, "plain"))

        context = ssl.create_default_context() # Create a secure SSL context

        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as server:
            server.login(sender_email, password)
            server.send_message(message)
        print("Email sent successfully!")
        return True
    except smtplib.SMTPAuthenticationError:
        print("SMTP Authentication Error: Check your username and password or app password/less secure app access settings.")
        return False
    except Exception as e:
        print(f"An error occurred while sending email: {e}")
        return False

In [ ]:
send_email_via_smtp("your email", email_data['to'], email_data['subject'],
                         email_data['body'], email_data['SMTP_SERVER'], 465, "your_app_password")

Email sent successfully!


True